<a href="https://colab.research.google.com/github/plaban1981/Langchain_usecases/blob/main/Summarize_YouTube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# From Youtube to Audio summaries
In this notebook let's use Whisper and ChatGPT to summarize a Youtube video

## 1.Downloading Youtube videos
Note: The legality of this depends on the geography

In [ ]:
!pip install pytube -q
from pytube import YouTube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.1 MB/s eta 0:00:00


In [ ]:
url = "https://www.youtube.com/watch?v=AyXITLa6Boc" #short
#url = "https://www.youtube.com/watch?v=3x4zM7O8pZI" #long
yt = YouTube(url)
audio = yt.streams.filter(only_audio=True).first().download()

In [ ]:
#Test if the Audio plays before we use whisper
from IPython.display import Audio
Audio(audio, rate = 22050)

## 2. Trancribe Audio with Whisper

In [ ]:
#First we will setup whisper
! pip install git+https://github.com/openai/whisper.git -q
import whisper
model = whisper.load_model("medium")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.9 MB/s eta 0:00:00


100%|█████████████████████████████████████| 1.42G/1.42G [00:29<00:00, 51.1MiB/s]


In [ ]:
transcript = model.transcribe(audio,fp16=False)['text']

In [ ]:
transcript[:110]

" Thank you very much. And it's great to be with a very good friend of mine, the Prime Minister of Pakistan. We"

## 3. Clean the sentences and summarize with LLMs

In [ ]:
#First Langchain text, by splitting a large audio file into smaller documents
!pip install langchain
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.9 MB/s eta 0:00:00


In [ ]:
text_splitter = CharacterTextSplitter()
texts = text_splitter.split_text(transcript)
docs = [Document(page_content=t) for t in texts[:3]]

In [ ]:
#Second, we will setup Summarize chain
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate

prompt_template = """Write an elegant summary in English without oversimplification of the following:{text}"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])

In [ ]:
#Third, we will use ChatGPT for this summarizing
!pip install openai

import os
os.environ["OPENAI_API_KEY"] = ""

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0.0)

chain = load_summarize_chain(llm, chain_type="stuff", prompt=PROMPT)

In [ ]:
chain.run(docs)

'President Trump met with the Prime Minister of Pakistan to discuss trade and other important issues, including the situation in Kashmir and the relationship between Pakistan and India. Both leaders expressed their interest in peace in Afghanistan and an orderly transition with talks between the Taliban and the government. The President also addressed the possibility of imposing auto tariffs on imports from Europe, stating that if a deal cannot be made, strong consideration will be given to taking action. Overall, the meeting was seen as a positive step in strengthening the relationship between the US and Pakistan.'

## Putting it all together in one function

In [ ]:
def summarize_yt(url, llm):
  yt = YouTube(url)
  audio = yt.streams.filter(only_audio=True).first().download()
  print("Audio downloaded")

  transcript = model.transcribe(audio,fp16=False)['text']
  print(transcript)

  text_splitter = CharacterTextSplitter()
  texts = text_splitter.split_text(transcript)
  docs = [Document(page_content=t) for t in texts[:3]]
  print("Text split")

  result = chain.run(docs)
  return result

In [ ]:
text = summarize_yt("https://www.youtube.com/watch?v=QPyXSU-TgVI", llm)
text

Audio downloaded
 இன்று காலையில் எழுந்துவிட்டு, குளித்து, மூடித்து விட்டு, கொயலுக்கு போகப் போறுகிறேன் கடவுளே ஏன்னா, என்ன அங்க? ஏய், என்ன கிண்டலா, புளியேடு, ராதியே, நாங்கள் எல்லாம் தேக்கு, யாரையும் பாப்பா, பழைய வெள்ளையத்தைவா, பாஞ்சா அளம் குருச்சியில் வெள்ளையனா, மெச்சினேன் வீராங்கனா, மெச்சினேன் என்ன觉்வர், உதவரா? வாங்க, வட்டம்போட்டு நிண்கிற்குள்ளான். �Does it not? ��ன்டக these untide, swimming cock-slinger binary To corral the one with a drop and also a moot After meeting them, you 144% We are only happy because you have come
Text split


'The passage is a mix of Tamil and English and appears to be a stream of consciousness thought process. The speaker is questioning why they feel the need to go to the forest and is pondering various things such as the color white, old memories, and asking for help. The passage ends with a statement about being happy because someone has arrived. The meaning behind the passage is unclear and may require further context to fully understand.'